In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

# 1. tensorflow v2.xx에서 v1사용하기

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  # tensorflow V2 비활성화하고 v1활성화
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


## Tensorflow
- 데이터 흐름 그래프(tensor 객체의 흐름)을 사용하는 수치 계산 라이브러리
- 그래프는 node(연산)와 edge로 구성
- sess = tf.Session()을 이용해서 실행 환경
- sess.run()을 통해서 실행 결과를 확인
- https://chaaany.tistory.com/306

In [8]:
# 1단계 : tensor(상수 노드 하나) 정의
node1 = tf.constant('Hello, Tensorflow')
# 2단계 : 세션(연산을 실행하는 환경) 생성
sess = tf.Session()
# 3단계 : 실제 실행
print(sess.run(node1))
print(sess.run(node1).decode())

b'Hello, Tensorflow'
Hello, Tensorflow


In [11]:
# 간단한 연산 tensor 그래프
# 1. 그래프 정의
node1 = tf.constant(10, dtype=tf.float16)
node2 = tf.constant(20, dtype=tf.float16)
node3 = tf.add(node1, node2)
# 2. 세션 생성
sess = tf.Session()
# 3. 세션 실행 & 결과
print(sess.run([node1, node2, node3]))

[10.0, 20.0, 30.0]


In [12]:
# 타입변경
import numpy as np
node1 = tf.constant(np.array([1,2,3]), dtype=tf.int16)
node2 = tf.cast(node1, dtype=tf.float16)
sess = tf.Session()
print(sess.run([node1, node2]))

[array([1, 2, 3], dtype=int16), array([1., 2., 3.], dtype=float16)]


In [13]:
# 평균값 계산 : tf.reduce_mean()
data = np.array([1.,2.,3.,4.])
m = tf.reduce_mean(data)
sess = tf.Session()
print(sess.run(m))

2


In [14]:
# tf.random_normal([size]): 평균이 0이고 표준편차가 1인 난수 size개 발생
w = tf.random_normal([3])
sess = tf.Session()
print(sess.run(w))

[-1.202876    1.8661578   0.52913326]


In [15]:
# 변수 노드 # 계속 미분해서 값을 변경해야해서
w = tf.Variable(tf.random_normal([1]))
sess = tf.Session()
sess.run(tf.global_variables_initializer())  # 변수 초기화
sess.run(w)

array([0.14914541], dtype=float32)

# 2. tensorflow v1을 이용한 회귀분석 구현
## 2.1 독립변수 x가 1개, 타겟(종속)변수 y가 1개

In [16]:
# tensor 그래프 정의
# 데이터 셋 확보
x = np.array([1,2,3])
y = np.array([2,3,4])
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis(가설) 모델의 예측값: 결과는 numpy 배열
H = W * x + b
# cost function (손실함수 : mse)  예측치와 실측치의 차이를 제곱하고 평균
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프. 곡선위 미분값이 0이 되는 방향
(경사하강법 GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습(v2에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b])
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))
# x값이 고정되어 있어 predict를 못한다.

0번째 cost:3.5727298259735107, W:[0.23041007], b:[0.9799147]
300번째 cost:0.0024215641897171736, W:[0.94298375], b:[1.1296115]
600번째 cost:0.0005713852006010711, W:[0.9723041], b:[1.0629592]
900번째 cost:0.00013482695794664323, W:[0.98654646], b:[1.0305831]
1200번째 cost:3.180988642270677e-05, W:[0.9934652], b:[1.0148551]
1500번째 cost:7.506545898650074e-06, W:[0.9968256], b:[1.0072162]
1800번째 cost:1.7713581428324687e-06, W:[0.99845785], b:[1.0035056]
2100번째 cost:4.1822889329523605e-07, W:[0.9992507], b:[1.0017033]
2400번째 cost:9.895565966644426e-08, W:[0.9996356], b:[1.0008284]
2700번째 cost:2.3540005500422012e-08, W:[0.99982226], b:[1.000404]
3000번째 cost:5.7308633394370645e-09, W:[0.9999124], b:[1.0001993]
3300번째 cost:1.4198491271599778e-09, W:[0.9999565], b:[1.0000993]
3600번째 cost:3.79410863837748e-10, W:[0.99997765], b:[1.0000514]
3900번째 cost:8.733517381509515e-11, W:[0.9999891], b:[1.0000248]
4200번째 cost:8.733517381509515e-11, W:[0.9999891], b:[1.0000248]
4500번째 cost:8.733517381509515e-11, W:[0

In [18]:
# 최종적으로 나온 회귀식 H=W*x + b
W_, b_ = sess.run([W,b])

(array([0.9999891], dtype=float32), array([1.0000248], dtype=float32))

In [19]:
def predict(x):
    return W_[0] * x + b_[0]

In [20]:
predict(5)

5.999970257282257

## 2.2 predict을 위한 placeholder이용
- placeholder : 외부에서 데이터를 입력받을 수 있는 노드
- H = W * x + b

In [22]:
x = tf.placeholder(dtype=tf.float32) # x를 외부에서 넣어 줄 수 있도록, x = np.array([1,2,3])는 고정된 값
H = 1*x + 1
sess = tf.Session()
sess.run([x,H], feed_dict={x : np.array([40,50])})

[array([40., 50.], dtype=float32), array([41., 51.], dtype=float32)]

In [23]:
# tensor 그래프 정의
# 데이터 셋 확보
x_data = np.array([1,2,3])
y_data = np.array([2,3,4])
# placeholder 설정(x,y)
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight와 bias
W = tf.Variable(tf.random.normal([1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')
# Hat, Hypothesis(가설) 모델의 예측값: 결과는 numpy 배열
H = W * x + b
# cost function (손실함수 : mse)  예측치와 실측치의 차이를 제곱하고 평균
cost = tf.reduce_mean(tf.square(H-y))
'''
학습목적 : cost가 최소가 되는 W, b를 찾는 것
cost 함수는 2차함수이므로 곡선 그래프. 곡선위 미분값이 0이 되는 방향
(경사하강법 GradientDescent)
'''
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 생성
sess = tf.Session()
# W, b 초기화
sess.run(tf.global_variables_initializer())
# 6000번 학습(v2에서의 fit함수)
for step in range(6001):
    _, cost_val, W_val, b_val = sess.run([train, cost, W, b],
                                        feed_dict={x:x_data,
                                                   y:y_data})
    if step%300==0:
        print("{}번째 cost:{}, W:{}, b:{}".format(step, cost_val, W_val, b_val))

0번째 cost:19.58757209777832, W:[-0.7307174], b:[0.7882529]
300번째 cost:0.007212432101368904, W:[0.9016009], b:[1.2236844]
600번째 cost:0.0017018363578245044, W:[0.95220196], b:[1.1086562]
900번째 cost:0.0004015662707388401, W:[0.9767818], b:[1.0527805]
1200번째 cost:9.47499429457821e-05, W:[0.9887217], b:[1.0256381]
1500번째 cost:2.2357911802828312e-05, W:[0.99452144], b:[1.012454]
1800번째 cost:5.275508101476589e-06, W:[0.9973387], b:[1.0060498]
2100번째 cost:1.2450217354853521e-06, W:[0.9987071], b:[1.002939]
2400번째 cost:2.940217029845371e-07, W:[0.99937165], b:[1.0014282]
2700번째 cost:6.957734655088643e-08, W:[0.99969435], b:[1.0006948]
3000번째 cost:1.6551553017052356e-08, W:[0.9998507], b:[1.0003388]
3300번째 cost:4.0335712547800995e-09, W:[0.9999267], b:[1.0001674]
3600번째 cost:9.71093983004323e-10, W:[0.9999639], b:[1.000082]
3900번째 cost:2.665766773990441e-10, W:[0.9999813], b:[1.0000429]
4200번째 cost:8.710306781400945e-11, W:[0.9999891], b:[1.0000248]
4500번째 cost:8.710306781400945e-11, W:[0.9999891

In [26]:
# 예측하기
sess.run(H, feed_dict={x:5})

array([5.9999704], dtype=float32)

## 2.3 scale 이 다른 데이터들의 회귀분석 구현(scale 조정x)

In [30]:
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5,15,68,80,95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight & bias
W = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))
# Hypothesis
H = W * x + b
# cost(손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)
# 세션 & 변수(W, b) 초기화 
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 6001):
    _, cost_val = sess.run([train, cost],feed_dict={x:x_data, y:y_data})
    if step%300==0:
        print("{}/6000번째 cost:{}".format(step+1, cost_val))

301/6000번째 cost:79.15251159667969
601/6000번째 cost:79.13980865478516
901/6000번째 cost:79.13945770263672
1201/6000번째 cost:79.13946533203125
1501/6000번째 cost:79.13945770263672
1801/6000번째 cost:79.13945007324219
2101/6000번째 cost:79.13944244384766
2401/6000번째 cost:79.13946533203125
2701/6000번째 cost:79.13946533203125
3001/6000번째 cost:79.13946533203125
3301/6000번째 cost:79.13946533203125
3601/6000번째 cost:79.13946533203125
3901/6000번째 cost:79.13946533203125
4201/6000번째 cost:79.13946533203125
4501/6000번째 cost:79.13946533203125
4801/6000번째 cost:79.13946533203125
5101/6000번째 cost:79.13946533203125
5401/6000번째 cost:79.13946533203125
5701/6000번째 cost:79.13946533203125
6001/6000번째 cost:79.13946533203125


## 2.4 scale이 다른 데이터들의 회귀분석 구현(scale 조정O)
### scale 조정방법 : 모든 데이터를 일정범위내로 조정
- normalization(정규화) : 모든 데이터를 0~1 사이로 조정 (최소값0, 최대값1이 되게)                      
    *                   X -Xmin
    * normalization = ─────────────
                       Xmax - Xmin
    * 위의 식보다 라이브러리 추천(sklearn.preprocessing.MinMaxScaler)
- standardization(표준화) : 데이터의 평균을 0, 표준편차를 1로 조정
    *                     X - Xmean
    * standardization = ──────────────
    *                    Xstd(표준편차)
    * 위의 식보다 라이브러리 추천(sklearn.preprocessing.StandardScaler)

In [36]:
# 라이브러리 쓰지 않고 정규화
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])
#x_data.min(), x_data.max(), x_data.mean(), x_data.std()
norm_scaled_x_data = (x_data - x_data.min()) / (x_data.max() - x_data.min())
norm_scaled_y_data = (y_data - y_data.min()) / (y_data.max() - y_data.min())

array([0.        , 0.11111111, 0.7       , 0.83333333, 1.        ])

In [43]:
# 라이브러리를 사용하여 정규화
x_data = np.array([1,2,5,8,10]).reshape(-1,1)
y_data = np.array([5,15,68,80,95]).reshape(-1,1)
from sklearn.preprocessing import MinMaxScaler   # MinMaxScaler - 변환모델을 만드는 클래스
scaler_x = MinMaxScaler()  # 독립변수 X를 정규화 시킬 객체
scaler_x.fit(x_data)
norm_scaled_x_data = scaler_x.transform(x_data)  

scaler_y = MinMaxScaler()  # 종속(타겟)변수 X를 정규화 시킬 객체
norm_scaled_y_data = scaler_y.fit_transform(y_data)
norm_scaled_y_data

array([[0.        ],
       [0.11111111],
       [0.7       ],
       [0.83333333],
       [1.        ]])

In [46]:
# 라이브러리 쓰지 않고 표준화하기
x_data = np.array([1,2,5,8,10])
y_data = np.array([5,15,68,80,95])
stan_scaled_x_data = (x_data - x_data.mean()) / x_data.std()
stan_scaled_y_data = (y_data - y_data.mean()) / y_data.std()
np.column_stack([x_data, norm_scaled_x_data, stan_scaled_x_data])

array([[ 1.        ,  0.        , -1.22474487],
       [ 2.        ,  0.11111111, -0.93313895],
       [ 5.        ,  0.44444444, -0.05832118],
       [ 8.        ,  0.77777778,  0.81649658],
       [10.        ,  1.        ,  1.39970842]])

In [47]:
np.column_stack([y_data, norm_scaled_y_data, stan_scaled_y_data])

array([[ 5.        ,  0.        , -1.32373476],
       [15.        ,  0.11111111, -1.04563922],
       [68.        ,  0.7       ,  0.42826713],
       [80.        ,  0.83333333,  0.76198177],
       [95.        ,  1.        ,  1.17912508]])

In [48]:
# 라이브러리를 사용하여 표준화
x_data = np.array([1,2,5,8,10]).reshape(-1,1)
y_data = np.array([5,15,68,80,95]).reshape(-1,1)

from sklearn.preprocessing import StandardScaler   
scaler_x = StandardScaler()  # 독립변수 X를 표준화 시킬 객체
stan_scaled_x_data = scaler_x.fit_transform(x_data)  

scaler_y = StandardScaler()  # 종속(타겟)변수 X를 표준화 시킬 객체
stan_scaled_y_data = scaler_y.fit_transform(y_data)

In [50]:
# 스케일 조정된 데이터를 다시 복구 : inverse_transform()이용
scaler_y.inverse_transform(stan_scaled_y_data)

array([[ 5.],
       [15.],
       [68.],
       [80.],
       [95.]])

In [56]:
x_data = np.array([1, 2, 5, 8, 10])
y_data = np.array([5,15,68,80,95])
# placeholder 설정
x = tf.placeholder(dtype=tf.float32)
y = tf.placeholder(dtype=tf.float32)
# weight & bias
W = tf.Variable(tf.random.normal([1]))
b = tf.Variable(tf.random.normal([1]))
# Hypothesis
H = W * x + b
# cost(손실함수)
cost = tf.reduce_mean(tf.square(H-y))
# 경사하강법
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train = optimizer.minimize(cost)
# 세션 & 변수(W, b) 초기화 
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 학습
for step in range(1, 6001):
    _, cost_val = sess.run([train, cost],feed_dict={x:norm_scaled_x_data, y:norm_scaled_y_data})
    if step%300==0:
        print("{}/6000번째 cost:{}".format(step, cost_val))

300/6000번째 cost:0.2995605170726776
600/6000번째 cost:0.09865036606788635
900/6000번째 cost:0.050576142966747284
1200/6000번째 cost:0.037094078958034515
1500/6000번째 cost:0.03170476481318474
1800/6000번째 cost:0.02843521162867546
2100/6000번째 cost:0.02590896561741829
2400/6000번째 cost:0.023783186450600624
2700/6000번째 cost:0.02195051871240139
3000/6000번째 cost:0.02036048285663128
3300/6000번째 cost:0.01897870935499668
3600/6000번째 cost:0.01777736283838749
3900/6000번째 cost:0.01673276722431183
4200/6000번째 cost:0.015824463218450546
4500/6000번째 cost:0.015034650452435017
4800/6000번째 cost:0.014347895979881287
5100/6000번째 cost:0.013750709593296051
5400/6000번째 cost:0.013231446966528893
5700/6000번째 cost:0.01277991570532322
6000/6000번째 cost:0.012387292459607124
